# Preparación del dataset de distancias final

En este documento se procede a la carga y limpieza del dataset generado en el documento *calculo_distancias.ipynb* para generar el dataset final utilizado en el algoritmo de ruta mínima. Se recuerda que el dataset inicial para este documento (*df_grafo_desc_completo*) cuenta con la siguiente estructura: 

- **distancia:** Distancia del trayecto en km.
- **duración:** Duración del trayecto en segundos.
- **origen:** Coordenadas del punto de origen del trayecto en formato *(latitud,longitud)*
- **destino:** Coordenadas del punto de destino del trayecto en formato *(latitud,longitud)*
- **denom_ini:** Denominación del punto de origen del trayecto.
- **denom_fin:** Denominación del punto de destino del trayecto.

## 1. Carga de ficheros iniciales y librerías

Se cargan en primer lugar los ficheros iniciales y librerías.

In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
import re

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
#r'C:\Users\Usuario\Desktop\Master UOC\Master ciencia de datos\TFM\matriz_destinos.csv'
df_grafo_desc = pd.read_csv(r'C:\Users\Usuario\Desktop\Master UOC\Master ciencia de datos\TFM\df_grafo_desc_completo.csv',
                        delimiter=",")
df_pc_cat = pd.read_csv(r'C:\Users\Usuario\Desktop\Master UOC\Master ciencia de datos\TFM\
Estacions_de_rec_rrega_per_a_vehicle_el_ctric_a_Catalunya.csv',
                        delimiter=",")

Antes de ejecutar el algoritmo de ruta mínima, es necesario generar un dataset final,limpio y utilizable para con este objetivo.

## 2. Limpieza y enriquecimiento del Dataset

### 2.1 Cruce de información de puntos de origen y destino

Se realiza inicialmente un cruce de la información de los puntos de origen del grafo con toda la información sobre los mismos, a través del campo de designación descriptiva.Se seleccionan únicamente los campos relevantes para el análisis. Se trata de las coordenadas del punto, tipo de cargador disponible, vehículo para el que está disponible este cargador, distancia en km y duración del trayecto en segundos.

In [5]:
df_grafo_desc_ini = pd.merge(df_grafo_desc, df_pc_cat,how = 'left', left_on='denom_ini'
                             ,right_on='DESIGNACIÓ-DESCRIPTIVA')

df_grafo_desc_ini = df_grafo_desc_ini.iloc[:, [2,4,8,17,19,21,3,0,1,5]]
df_grafo_desc_ini.columns = ['punto_ini', 'denom_ini','tipo_carg_ini','provincia_ini','municipio_ini','tipo_vehic_ini',
                         'punto_fin','distancia','duracion','denom_fin']

In [6]:
df_grafo_desc_ini

,punto_ini,denom_ini,tipo_carg_ini,provincia_ini,municipio_ini,tipo_vehic_ini,punto_fin,distancia,duracion,denom_fin
0,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(40.80762, 0.52007)",164 km,6091,EdRsR Tortosa (AC22kW) PIRVEC-24
1,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.38455, 2.1376713)",15.1 km,1150,HOTEL NH NUMANCIA
2,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.394009000000004, 2.115265)",13.8 km,933,B:SM 20 - Marques de Mulhacen
3,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.4047, 2.1896)",22.5 km,1644,B:SM 24 - Ona Glòries
4,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.66519, 1.86011)",52.1 km,2512,EdRsR Sant Vicenç de Castellet (AC22kW)
...,...,...,...,...,...,...,...,...,...,...
87699,"(41.613690000000005, 2.3454900000000003)",CC La Roca Village Aparcament (11kW),semiRAPID i NORMAL,Barcelona,La Roca del Vallès,cotxe i moto,"(41.389255, 2.161321)",35.3 km,2368,28 AjBCN-Endesa-Aragó-Pl.Letamendi
87700,"(41.613690000000005, 2.3454900000000003)",CC La Roca Village Aparcament (11kW),semiRAPID i NORMAL,Barcelona,La Roca del Vallès,cotxe i moto,"(41.56064, 2.0059)",40.8 km,1960,Aparcament Plaça Progrés
87701,"(41.372299, 2.153134)",B:SM 35 - Rius i Taulet,NORMAL,Barcelona,Barcelona,cotxe i moto,"(41.389255, 2.161321)",3.2 km,744,28 AjBCN-Endesa-Aragó-Pl.Letamendi
87702,"(41.372299, 2.153134)",B:SM 35 - Rius i Taulet,NORMAL,Barcelona,Barcelona,cotxe i moto,"(41.56064, 2.0059)",41.0 km,2285,Aparcament Plaça Progrés


Se realiza el mismo proceso para los datos del punto de destino.

In [7]:
df_grafo_desc_fin = pd.merge(df_grafo_desc_ini, df_pc_cat,how = 'left', left_on='denom_fin'
                             ,right_on='DESIGNACIÓ-DESCRIPTIVA')

In [8]:
df_grafo_desc_fin

,punto_ini,denom_ini,tipo_carg_ini,provincia_ini,municipio_ini,tipo_vehic_ini,punto_fin,distancia,duracion,denom_fin,...,POTENCIA,TIPUS DE CORRENT,INDENTIFICADOR,ADREÇA,PROVINCIA,CODIPROV,MUNICIPI,NPLACES ESTACIÓ,TIPUS VEHICLE,Columna amb georeferència
0,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(40.80762, 0.52007)",164 km,6091,EdRsR Tortosa (AC22kW) PIRVEC-24,...,22.0,AC,PIRVEC-24,"Carrer d'Alcanyís, 9",Tarragona,43,Tortosa,NaN,NaN,POINT (0.52007 40.80762)
1,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.38455, 2.1376713)",15.1 km,1150,HOTEL NH NUMANCIA,...,NaN,NaN,NaN,"Carrer de Numància, 74",Barcelona,8,Barcelona,1,NaN,POINT (2.1376712999999654 41.38455)
2,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.394009000000004, 2.115265)",13.8 km,933,B:SM 20 - Marques de Mulhacen,...,NaN,NaN,BSM015,"Carrer de Marquès de Mulhacén, 49-51",Barcelona,8,Barcelona,6+2,cotxe i moto,POINT (2.115265 41.394009)
3,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.4047, 2.1896)",22.5 km,1644,B:SM 24 - Ona Glòries,...,NaN,NaN,BSM037,"Carrer de la Ciutat de Granada, 173-175",Barcelona,8,Barcelona,4,cotxe,POINT (2.1896 41.4047)
4,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.66519, 1.86011)",52.1 km,2512,EdRsR Sant Vicenç de Castellet (AC22kW),...,22.0,AC,NaN,"Plaça Generalitat, 4",Barcelona,8,Sant Vicenç de Castellet,2,NaN,POINT (1.86011 41.66519)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87983,"(41.613690000000005, 2.3454900000000003)",CC La Roca Village Aparcament (11kW),semiRAPID i NORMAL,Barcelona,La Roca del Vallès,cotxe i moto,"(41.389255, 2.161321)",35.3 km,2368,28 AjBCN-Endesa-Aragó-Pl.Letamendi,...,NaN,NaN,LC008,"Plaça del Dr. Letamendi, 30-33",Barcelona,8,Barcelona,2,moto,POINT (2.161321 41.389255)
87984,"(41.613690000000005, 2.3454900000000003)",CC La Roca Village Aparcament (11kW),semiRAPID i NORMAL,Barcelona,La Roca del Vallès,cotxe i moto,"(41.56064, 2.0059)",40.8 km,1960,Aparcament Plaça Progrés,...,NaN,NaN,NaN,"Plaça del Progrés, 8",Barcelona,8,Terrassa,4,cotxe i moto,POINT (2.0059 41.56064)
87985,"(41.372299, 2.153134)",B:SM 35 - Rius i Taulet,NORMAL,Barcelona,Barcelona,cotxe i moto,"(41.389255, 2.161321)",3.2 km,744,28 AjBCN-Endesa-Aragó-Pl.Letamendi,...,NaN,NaN,LC008,"Plaça del Dr. Letamendi, 30-33",Barcelona,8,Barcelona,2,moto,POINT (2.161321 41.389255)
87986,"(41.372299, 2.153134)",B:SM 35 - Rius i Taulet,NORMAL,Barcelona,Barcelona,cotxe i moto,"(41.56064, 2.0059)",41.0 km,2285,Aparcament Plaça Progrés,...,NaN,NaN,NaN,"Plaça del Progrés, 8",Barcelona,8,Terrassa,4,cotxe i moto,POINT (2.0059 41.56064)


Se reorganizan las columnas de salida para mantener los mismos campos tanto para origen como para destino. Los campos de punto de origen tienen el sufijo '_ini' mientras que los campos de destino tienen el prefijo '_fin' 

In [9]:
df_grafo_desc_fin.columns

Index(['punto_ini', 'denom_ini', 'tipo_carg_ini', 'provincia_ini',
       'municipio_ini', 'tipo_vehic_ini', 'punto_fin', 'distancia', 'duracion',
       'denom_fin', 'PROMOTOR-GESTOR', 'ACCES', 'TIPUS VELOCITAT',
       'TIPUS CONNEXIÓ', 'LATITUD', 'LONGITUD', 'DESIGNACIÓ-DESCRIPTIVA',
       'POTENCIA', 'TIPUS DE CORRENT', 'INDENTIFICADOR', 'ADREÇA', 'PROVINCIA',
       'CODIPROV', 'MUNICIPI', 'NPLACES ESTACIÓ', 'TIPUS VEHICLE',
       'Columna amb georeferència'],
      dtype='object')

In [10]:
df_grafo_desc_fin = df_grafo_desc_fin.iloc[:, [0,1,2,3,4,5,6,9,12,21,23,25,7,8]]
df_grafo_desc_fin.columns = ['punto_ini', 'denom_ini','tipo_carg_ini','provincia_ini','municipio_ini','tipo_vehic_ini',
                        'punto_fin','denom_fin'
                             ,'tipo_carg_fin','provincia_fin','municipio_fin','tipo_vehic_fin','distancia','duracion']

In [11]:
df_grafo_desc_fin

,punto_ini,denom_ini,tipo_carg_ini,provincia_ini,municipio_ini,tipo_vehic_ini,punto_fin,denom_fin,tipo_carg_fin,provincia_fin,municipio_fin,tipo_vehic_fin,distancia,duracion
0,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(40.80762, 0.52007)",EdRsR Tortosa (AC22kW) PIRVEC-24,semiRAPID,Tarragona,Tortosa,NaN,164 km,6091
1,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.38455, 2.1376713)",HOTEL NH NUMANCIA,NORMAL,Barcelona,Barcelona,NaN,15.1 km,1150
2,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.394009000000004, 2.115265)",B:SM 20 - Marques de Mulhacen,NORMAL,Barcelona,Barcelona,cotxe i moto,13.8 km,933
3,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.4047, 2.1896)",B:SM 24 - Ona Glòries,NORMAL,Barcelona,Barcelona,cotxe,22.5 km,1644
4,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.66519, 1.86011)",EdRsR Sant Vicenç de Castellet (AC22kW),semiRAPID,Barcelona,Sant Vicenç de Castellet,NaN,52.1 km,2512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87983,"(41.613690000000005, 2.3454900000000003)",CC La Roca Village Aparcament (11kW),semiRAPID i NORMAL,Barcelona,La Roca del Vallès,cotxe i moto,"(41.389255, 2.161321)",28 AjBCN-Endesa-Aragó-Pl.Letamendi,NORMAL,Barcelona,Barcelona,moto,35.3 km,2368
87984,"(41.613690000000005, 2.3454900000000003)",CC La Roca Village Aparcament (11kW),semiRAPID i NORMAL,Barcelona,La Roca del Vallès,cotxe i moto,"(41.56064, 2.0059)",Aparcament Plaça Progrés,NORMAL,Barcelona,Terrassa,cotxe i moto,40.8 km,1960
87985,"(41.372299, 2.153134)",B:SM 35 - Rius i Taulet,NORMAL,Barcelona,Barcelona,cotxe i moto,"(41.389255, 2.161321)",28 AjBCN-Endesa-Aragó-Pl.Letamendi,NORMAL,Barcelona,Barcelona,moto,3.2 km,744
87986,"(41.372299, 2.153134)",B:SM 35 - Rius i Taulet,NORMAL,Barcelona,Barcelona,cotxe i moto,"(41.56064, 2.0059)",Aparcament Plaça Progrés,NORMAL,Barcelona,Terrassa,cotxe i moto,41.0 km,2285


### 2.2 Formato de latitud y longitud

El campo que incluye las coordenadas geográficas de cada uno de los puntos, tanto origen como destino, está definido inicialment en formato (latitud,longitud). Para poder trabajar más fácilmente con las coordenadas se crearán nuevos a partir de los anteriores, cada uno de ellos incluyendo latitud y longitud de forma separada, tanto para el punto origen como el punto destino.

In [12]:
#Mediante expresiones regulares, se separan latitud y longitud como elementos de una tupla.
punto_ini = []
for index, row in df_grafo_desc_fin.iterrows():
    punto_ini.append(re.sub('[()]', '', df_grafo_desc_fin["punto_ini"][index]).split(', '))

In [13]:
#Se generan listas independientes con latitud y longitud
lat_ini = []
lon_ini = []
for i in punto_ini:
    lat_ini.append(i[0])
    lon_ini.append(i[1])

In [14]:
punto_fin = []
for index, row in df_grafo_desc_fin.iterrows():
    punto_fin.append(re.sub('[()]', '', df_grafo_desc_fin["punto_fin"][index]).split(', '))

In [15]:
lat_fin = []
lon_fin = []
for i in punto_fin:
    lat_fin.append(i[0])
    lon_fin.append(i[1])

In [16]:
#Se definen nuevos campos para las coordenadas geográficas
df_grafo_desc_fin["lat_ini"] = lat_ini
df_grafo_desc_fin["lon_ini"] = lon_ini
df_grafo_desc_fin["lat_fin"] = lat_fin
df_grafo_desc_fin["lon_fin"] = lon_fin

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [18]:
df_grafo_desc_fin

,punto_ini,denom_ini,tipo_carg_ini,provincia_ini,municipio_ini,tipo_vehic_ini,punto_fin,denom_fin,tipo_carg_fin,provincia_fin,municipio_fin,tipo_vehic_fin,distancia,duracion,lat_ini,lon_ini,lat_fin,lon_fin
0,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(40.80762, 0.52007)",EdRsR Tortosa (AC22kW) PIRVEC-24,semiRAPID,Tarragona,Tortosa,NaN,164 km,6091,41.321165,2.027702,40.80762,0.52007
1,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.38455, 2.1376713)",HOTEL NH NUMANCIA,NORMAL,Barcelona,Barcelona,NaN,15.1 km,1150,41.321165,2.027702,41.38455,2.1376713
2,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.394009000000004, 2.115265)",B:SM 20 - Marques de Mulhacen,NORMAL,Barcelona,Barcelona,cotxe i moto,13.8 km,933,41.321165,2.027702,41.394009000000004,2.115265
3,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.4047, 2.1896)",B:SM 24 - Ona Glòries,NORMAL,Barcelona,Barcelona,cotxe,22.5 km,1644,41.321165,2.027702,41.4047,2.1896
4,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.66519, 1.86011)",EdRsR Sant Vicenç de Castellet (AC22kW),semiRAPID,Barcelona,Sant Vicenç de Castellet,NaN,52.1 km,2512,41.321165,2.027702,41.66519,1.86011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87983,"(41.613690000000005, 2.3454900000000003)",CC La Roca Village Aparcament (11kW),semiRAPID i NORMAL,Barcelona,La Roca del Vallès,cotxe i moto,"(41.389255, 2.161321)",28 AjBCN-Endesa-Aragó-Pl.Letamendi,NORMAL,Barcelona,Barcelona,moto,35.3 km,2368,41.613690000000005,2.3454900000000003,41.389255,2.161321
87984,"(41.613690000000005, 2.3454900000000003)",CC La Roca Village Aparcament (11kW),semiRAPID i NORMAL,Barcelona,La Roca del Vallès,cotxe i moto,"(41.56064, 2.0059)",Aparcament Plaça Progrés,NORMAL,Barcelona,Terrassa,cotxe i moto,40.8 km,1960,41.613690000000005,2.3454900000000003,41.56064,2.0059
87985,"(41.372299, 2.153134)",B:SM 35 - Rius i Taulet,NORMAL,Barcelona,Barcelona,cotxe i moto,"(41.389255, 2.161321)",28 AjBCN-Endesa-Aragó-Pl.Letamendi,NORMAL,Barcelona,Barcelona,moto,3.2 km,744,41.372299,2.153134,41.389255,2.161321
87986,"(41.372299, 2.153134)",B:SM 35 - Rius i Taulet,NORMAL,Barcelona,Barcelona,cotxe i moto,"(41.56064, 2.0059)",Aparcament Plaça Progrés,NORMAL,Barcelona,Terrassa,cotxe i moto,41.0 km,2285,41.372299,2.153134,41.56064,2.0059


In [15]:
df_grafo_desc_fin.head()

,punto_ini,denom_ini,tipo_carg_ini,provincia_ini,municipio_ini,tipo_vehic_ini,punto_fin,denom_fin,tipo_carg_fin,provincia_fin,municipio_fin,tipo_vehic_fin,distancia,duracion,lat_ini,lon_ini,lat_fin,lon_fin
0,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(40.80762, 0.52007)",EdRsR Tortosa (AC22kW) PIRVEC-24,semiRAPID,Tarragona,Tortosa,NaN,164 km,6091,41.321165,2.027702,40.807620,0.520070
1,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.38455, 2.1376713)",HOTEL NH NUMANCIA,NORMAL,Barcelona,Barcelona,NaN,15.1 km,1150,41.321165,2.027702,41.384550,2.137671
2,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.394009000000004, 2.115265)",B:SM 20 - Marques de Mulhacen,NORMAL,Barcelona,Barcelona,cotxe i moto,13.8 km,933,41.321165,2.027702,41.394009,2.115265
3,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.4047, 2.1896)",B:SM 24 - Ona Glòries,NORMAL,Barcelona,Barcelona,cotxe,22.5 km,1644,41.321165,2.027702,41.404700,2.189600
4,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.66519, 1.86011)",EdRsR Sant Vicenç de Castellet (AC22kW),semiRAPID,Barcelona,Sant Vicenç de Castellet,NaN,52.1 km,2512,41.321165,2.027702,41.665190,1.860110


### 2.2 Formato de distancias

Respecto al formato del campo 'distancia', la API de Google maps devuelve junto al valor la unidad de medida (km). La unidad se eliminará del campo para trabajar adecuadamente con valores numéricos.

In [16]:
df_grafo_desc_fin[['distancia', 'unidad']] = df_grafo_desc_fin['distancia'].str.split(' ', 1, expand=True)

In [17]:
df_grafo_desc_fin = df_grafo_desc_fin.drop(['unidad'], axis=1)

In [18]:
df_grafo_desc_fin.head()

,punto_ini,denom_ini,tipo_carg_ini,provincia_ini,municipio_ini,tipo_vehic_ini,punto_fin,denom_fin,tipo_carg_fin,provincia_fin,municipio_fin,tipo_vehic_fin,distancia,duracion,lat_ini,lon_ini,lat_fin,lon_fin
0,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(40.80762, 0.52007)",EdRsR Tortosa (AC22kW) PIRVEC-24,semiRAPID,Tarragona,Tortosa,NaN,164,6091,41.321165,2.027702,40.807620,0.520070
1,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.38455, 2.1376713)",HOTEL NH NUMANCIA,NORMAL,Barcelona,Barcelona,NaN,15.1,1150,41.321165,2.027702,41.384550,2.137671
2,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.394009000000004, 2.115265)",B:SM 20 - Marques de Mulhacen,NORMAL,Barcelona,Barcelona,cotxe i moto,13.8,933,41.321165,2.027702,41.394009,2.115265
3,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.4047, 2.1896)",B:SM 24 - Ona Glòries,NORMAL,Barcelona,Barcelona,cotxe,22.5,1644,41.321165,2.027702,41.404700,2.189600
4,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.66519, 1.86011)",EdRsR Sant Vicenç de Castellet (AC22kW),semiRAPID,Barcelona,Sant Vicenç de Castellet,NaN,52.1,2512,41.321165,2.027702,41.665190,1.860110


### 2.3 Filtrado de dataset por tipo de cargador

Una vez definidos los campos finales del dataset, se realizará un filtrado previo que resulta necesario para mantener la congruencia del análisis. Dado que este análisis se centra exclusivamente en coche elétrico, las motos eléctricas quedan excluidas del análisis. Por lo que todos los cargadores exclusivamente de moto, serán eliminados, tanto de origen como de destino (tipo_vehic_ini y tipo_vehic_fin). Para los casos en los que este campo no está informado, dado que el volumen de cargadores con toma para coche o moto, o exclusivo para coche son predominantes, se asumirá que son también válidos para coche y se mantendrán en el análisis.

In [19]:
df_grafo_desc_fin = df_grafo_desc_fin.loc[(df_grafo_desc_fin['tipo_vehic_ini']!='moto')]
df_grafo_desc_fin = df_grafo_desc_fin.loc[(df_grafo_desc_fin['tipo_vehic_fin']!='moto')]

In [20]:
df_grafo_desc_fin

,punto_ini,denom_ini,tipo_carg_ini,provincia_ini,municipio_ini,tipo_vehic_ini,punto_fin,denom_fin,tipo_carg_fin,provincia_fin,municipio_fin,tipo_vehic_fin,distancia,duracion,lat_ini,lon_ini,lat_fin,lon_fin
0,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(40.80762, 0.52007)",EdRsR Tortosa (AC22kW) PIRVEC-24,semiRAPID,Tarragona,Tortosa,NaN,164,6091,41.321165,2.027702,40.807620,0.520070
1,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.38455, 2.1376713)",HOTEL NH NUMANCIA,NORMAL,Barcelona,Barcelona,NaN,15.1,1150,41.321165,2.027702,41.384550,2.137671
2,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.394009000000004, 2.115265)",B:SM 20 - Marques de Mulhacen,NORMAL,Barcelona,Barcelona,cotxe i moto,13.8,933,41.321165,2.027702,41.394009,2.115265
3,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.4047, 2.1896)",B:SM 24 - Ona Glòries,NORMAL,Barcelona,Barcelona,cotxe,22.5,1644,41.321165,2.027702,41.404700,2.189600
4,"(41.321165, 2.027702)",CC ALDI,NORMAL,Barcelona,Viladecans,cotxe,"(41.66519, 1.86011)",EdRsR Sant Vicenç de Castellet (AC22kW),semiRAPID,Barcelona,Sant Vicenç de Castellet,NaN,52.1,2512,41.321165,2.027702,41.665190,1.860110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87979,"(42.426809999999996, 1.9292200000000002)",EdRR Puigcerdà (DC50kW AC43kW) PIRVEC-20,RAPID,Girona,Puigcerdà,NaN,"(41.372299, 2.153134)",B:SM 35 - Rius i Taulet,NORMAL,Barcelona,Barcelona,cotxe i moto,151,6972,42.426810,1.929220,41.372299,2.153134
87981,"(42.426809999999996, 1.9292200000000002)",EdRR Puigcerdà (DC50kW AC43kW) PIRVEC-20,RAPID,Girona,Puigcerdà,NaN,"(41.56064, 2.0059)",Aparcament Plaça Progrés,NORMAL,Barcelona,Terrassa,cotxe i moto,124,5497,42.426810,1.929220,41.560640,2.005900
87982,"(41.613690000000005, 2.3454900000000003)",CC La Roca Village Aparcament (11kW),semiRAPID i NORMAL,Barcelona,La Roca del Vallès,cotxe i moto,"(41.372299, 2.153134)",B:SM 35 - Rius i Taulet,NORMAL,Barcelona,Barcelona,cotxe i moto,52.0,2635,41.613690,2.345490,41.372299,2.153134
87984,"(41.613690000000005, 2.3454900000000003)",CC La Roca Village Aparcament (11kW),semiRAPID i NORMAL,Barcelona,La Roca del Vallès,cotxe i moto,"(41.56064, 2.0059)",Aparcament Plaça Progrés,NORMAL,Barcelona,Terrassa,cotxe i moto,40.8,1960,41.613690,2.345490,41.560640,2.005900


In [21]:
df_grafo_desc_fin.to_csv('df_grafo_desc_fin.csv',index=False)